###Import the required libraries

In [0]:

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import os
import zipfile
import random
import shutil
from shutil import copyfile
import csv


###Retrieve the required dataset

In [0]:

from keras_preprocessing.image import ImageDataGenerator

# Function to read data from the csv files
def get_data(filename):
    with open(filename) as training_file:
        image = []
        label = []
        stat = 0
        for row in csv.reader(training_file):
            if stat == 0:
                stat = 1
            else:
                label.append(float(row[0]))
                a = np.reshape(np.array(row[1:785]).astype(float), (28, 28))
                image.append(a)
        labels = np.array(label)
        images = np.array(image)
    return images, labels

# Define the paths to the datasets
path_sign_mnist_train = "sign_mnist_train.csv"
path_sign_mnist_test = "sign_mnist_test.csv"
training_images, training_labels = get_data(path_sign_mnist_train)
testing_images, testing_labels = get_data(path_sign_mnist_test)

# Print the size of the dataset
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

# Expected Output:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)


###Pre-process and augment the data

In [0]:

# Adding an extra dimension to the datasets
training_images = np.expand_dims(training_images, axis = 3)
testing_images = np.expand_dims(testing_images, axis = 3)

# Converting the data to categorical data, to use the "categorical_crossentropy" loss function
from tensorflow.keras.utils import to_categorical
training_lab = to_categorical(training_labels)
testing_lab = to_categorical(testing_labels)

# Pre-process and augment the training data
train_datagen = ImageDataGenerator(rescale = 1./255,
	                                 rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest')
train_generator = train_datagen.flow(training_images,
                                     training_lab,
                                     batch_size = 50)

# Pre-process the test data
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow(testing_images,
                                               testing_lab,
                                               batch_size = 50)

# Print the size of the dataset
print(training_images.shape)
print(testing_images.shape)
    
# Expected Output:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)


###Define the model

In [0]:

# Define the layers for the model
sl_model = tf.keras.models.Sequential ([
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (28, 28, 1), name = 'Conv1'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2), name = 'Pool1'),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', name = 'Conv2'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2), name = 'Pool2'),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', name = 'Conv3'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2), name = 'Pool3'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu', name = 'Dense4'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(25, activation = 'softmax', name = 'Output5')
])

# Define the optimizer and loss function for the model
sl_model.compile(loss = 'categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])


###Train the model

In [0]:

# Fit the model and store the intermediate outputs
history = sl_model.fit(train_generator,
                       epochs = 10,
                       steps_per_epoch = 550,
                       validation_data = validation_generator,
                       validation_steps = 144, 
                       verbose = 1)


###Plot the loss and accuracy of the model w.r.t the number of epochs

In [0]:

# Retrieve a list of results on training and test datasets for each training epoch
acc      = history.history['accuracy']
val_acc  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']

# Get the total number of epochs
epochs   = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot  (epochs, acc)
plt.plot  (epochs, val_acc)
plt.title ('Training and validation accuracy', color = "white")
plt.figure()

# Plot training and validation loss per epoch
plt.plot  (epochs, loss)
plt.plot  (epochs, val_loss)
plt.title ('Training and validation loss', color = "white")


###Get a summary of the model

In [0]:

# Print the summary of the model
sl_model.summary()
